# Main Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

In [198]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from utils.visualisations_helpers import *

set_visualization_style()

viridis = plt.get_cmap('viridis_r', 6)  # Extract 6 colors from reversed viridis

In [199]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from tableone import TableOne

# Load data from pickle
df_wave_7 = pd.read_pickle('data/wave7_data.pkl')
df_wave_7_part1 = pd.read_pickle('data/wave7_data_part1.pkl')
df_wave_7_part2 = pd.read_pickle('data/wave7_data_part2.pkl')

In [ ]:
df_wave_7.columns

In [ ]:
categorical_columns = df_wave_7.select_dtypes(include=['category']).columns
for col in categorical_columns:
    print(f"Categories for {col}: {df_wave_7[col].cat.categories}")
    print(f"Reference category for {col}: {df_wave_7[col].cat.categories[0]}")

## I/ Exploratory data analysis

### A) Summary Stats

I start by exploring some descriptive statistics, for all respondents, and grouped by whether they are UK natives or not. 

In [ ]:
columns = ['sf12mcs_dv','sex', 'age_dv', 'ethn_dv_recoded', 'hiqual_dv_recoded', 'years_since_arrival_binary', 'mreason', 'unemployed']
continuous = ['sf12mcs_dv','age_dv']
categorical = ['years_since_arrival_binary', 'sex','ethn_dv_recoded', 'hiqual_dv_recoded', 'mreason', 'unemployed']
groupby = 'imm_group'

# Define the mapping for renaming columns
rename = {
    'sf12mcs_dv': 'SF-12 MCS',
    'sex': 'Sex',
    'age_dv': 'Age',
    'ethn_dv_recoded': 'Ethnicity',
    'hiqual_dv_recoded': 'Highest Qualification',
    'jbstat_dv_recoded': 'Labour Force Status',
    'gor_dv_recoded': 'Government Office Region',
    'years_since_arrival_binary': 'Has lived in the UK for more than 15 years',
    'mreason': 'Reason for Migrating'
}

# Create the TableOne object with the renaming parameter
mytable = TableOne(df_wave_7, columns, categorical, continuous, groupby, rename=rename)

print(mytable.tabulate(tablefmt="github"))

# mytable.to_latex('tables/table_1.tex')

### B) Visualisations

In [ ]:
# ===============================================================
# === Mental health by immigration status & reason to migrate ===
# ===============================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
# Violin plot
sns.violinplot(data=df_wave_7, x='immigrant', y='sf12mcs_dv', ax=ax1)
ax1.set_title('Distribution of Mental Health Scores\nImmigrants vs UK-born', pad=20)
ax1.set_xlabel('Immigration Status')
ax1.set_ylabel('SF-12 Mental Component Score')

# Box plot by migration reason
sns.boxplot(data=df_wave_7[df_wave_7['immigrant']==1], 
            x='mreason', y='sf12mcs_dv',
            ax=ax2)
ax2.set_title('Mental Health Scores by Migration Reason', pad=20)
ax2.set_xlabel('Reason for Migration')
ax2.set_ylabel('SF-12 Mental Component Score')
ax2.tick_params(axis='x', rotation=45)

# Adjust layout
plt.tight_layout()

In [ ]:
# ========================================================
# === Mental health by years since arrival & ethnicity ===
# ========================================================
# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))

# Line plot: Average MH scores by years since arrival and ethnicity
sns.lineplot(
    data=df_wave_7[df_wave_7['immigrant']==1],
    x='years_since_arrival_cat',
    y='sf12mcs_dv',
    hue='ethn_dv_recoded',
    markers=True,
    marker='o',
    ax=ax1
)

ax1.set_title('Mental Health Scores by Years Since Immigration\nAcross Ethnic Groups', pad=20)
ax1.set_xlabel('Years Since Arrival')
ax1.set_ylabel('Average SF-12 Mental Component Score')
ax1.tick_params(axis='x', rotation=45)

# Add legend with better positioning
ax1.legend(title='Ethnic Group', bbox_to_anchor=(1.05, 1), loc='upper left')

# Bar plot: Population distribution
# Calculate the distribution of immigrants across time categories by ethnicity
immigrant_counts = df_wave_7[df_wave_7['immigrant']==1].groupby(['years_since_arrival_cat', 'ethn_dv_recoded']).size().unstack()

immigrant_counts.plot(
    kind='bar',
    stacked=True,
    ax=ax2
)

ax2.set_title('Distribution of Immigrants by Years Since Arrival\nAcross Ethnic Groups', pad=20)
ax2.set_xlabel('Years Since Arrival')
ax2.set_ylabel('Number of Immigrants')
ax2.tick_params(axis='x', rotation=45)
ax2.legend(title='Ethnic Group', bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust layout to prevent overlap
plt.tight_layout()

## II/ Examining the Healthy Immigrant Effect

In [ ]:
# MODEL 1: Mental Health of UK-born and immigrants
formula_m1 = 'sf12mcs_dv ~ immigrant'
model1 = ols(formula=formula_m1, data=df_wave_7_part1).fit(cov_type='HC3')

print(model1.summary())

The results of this baseline regression indicate at the absence of a statistically significant difference in mental health between UK-born and immigrants.

In [ ]:
# MODEL 2: Added controls
formula_m2 = 'sf12mcs_dv ~ immigrant + age_dv + pow(age_dv, 2) + ethn_dv_recoded + sex + hiqual_dv_recoded'
model2 = ols(formula=formula_m2, data=df_wave_7_part1).fit(cov_type='HC3')

print(model2.summary())

This regression with controls offers a more insightful picture of the difference in mental health between UK-born and immigrant respondents. The statistically significant negative coefficient for the 'UK-born' category gives support to the 'healthy immigrant' hypothesis. More specifically, immigrants exhibit, on average, a mental health score 0.69 higher than natives, controlling for ethnicity, age, education and occupational status.

The caveat is the low R-squared of this model (0.06), which indicates that the model does not explain very well variation in mental health. Although this can be partly explain by the complexity and diversity of factors that determine it, it is a rather concerning indicator of omitted variable bias.



A strong limitation of this model, and of many studies in the literature examining the mental health of immigrants, is the holistic grouping of immigrants to which the 'healthy' effect supposedly apply. It is likely that reasons for migration may affect the mental health of immigrants. 

Because the 'reason to migrate' variable does not apply to UK-born respondents, it is added with the intervention of an indicator variable. Specifically, the effect of this variable is only considered as part of an interaction with the main effect of being an immigrant on mental health in the model. Note that this is an exception to the general rule that terms should not be included as interactions without a main effect term. 

In [ ]:
# MODEL 3: Added interaction with reasons to migrate
formula_m3 = 'sf12mcs_dv ~ immigrant + immigrant:mreason + age_dv + pow(age_dv, 2) + sex + ethn_dv_recoded + hiqual_dv_recoded'
model3 = ols(formula=formula_m3, data=df_wave_7_part1).fit(cov_type='HC3')

print(model3.summary())

## III/ Examining the Years since Immigration Effect

Now, I reduce the sample to only immigrants. I ask whether time since arrival has an effect on the mental health of immigrants.

In [ ]:
# MODEL 4: Mental Health of UK-born and immigrants with years since arrival
formula_m4 = 'sf12mcs_dv ~  years_since_arrival_binary + ethn_dv_recoded'
model4 = ols(formula=formula_m4, data=df_wave_7_part2).fit(cov_type='HC3')

print(model4.summary())

In [ ]:
# MODEL 5: Mental Health of UK-born and immigrants with years since arrival
formula_m5 = 'sf12mcs_dv ~  years_since_arrival_binary + age_dv + pow(age_dv, 2) + ethn_dv_recoded + sex + hiqual_dv_recoded'
model5 = ols(formula=formula_m5, data=df_wave_7_part2).fit(cov_type='HC3')

print(model5.summary())

In [ ]:
# MODEL 6: Added interaction with ethnicity
formula_m6 = 'sf12mcs_dv ~  years_since_arrival_binary*ethn_dv_recoded + age_dv + pow(age_dv, 2) + sex + hiqual_dv_recoded'
model6 = ols(formula=formula_m6, data=df_wave_7_part2).fit(cov_type='HC3')

print(model6.summary())

## IV/ Testing OLS Assumptions

I now check the assumptions for the linear regressions. I test them for Model 3 and Model 6, because they are the fully specified models for each section of the analysis.

In [ ]:
# =================
# === Linearity ===
# =================
# For binary and categorical variables: 
# This assumption is trivially met, since the line of best fit is modeled as a straight line between pairs of conditional means

# For continuous variables (only age here): 
# Plot x vs. y
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
sns.scatterplot(data=df_wave_7, x='age_dv', y='sf12mcs_dv', ax=ax, color=viridis(3))
ax.set_xlabel('Age')
ax.set_ylabel('SF-12 Mental Component Score')

# Export figure to jpg
fig.savefig('figures/agevsMH.jpg', dpi=300)

In [ ]:
# Check residuals vs fitted values
# Combined Residuals vs Fitted Values plot for Model 3 and Model 6
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Model 3
sns.residplot(x=model3.fittedvalues, y=model3.resid, lowess=True, line_kws={'color': 'red', 'lw': 2}, ax=ax1, scatter_kws={'color': viridis(2)})
ax1.axhline(0, color='black', linestyle='--', linewidth=1)
ax1.set_xlabel("Fitted Values")
ax1.set_ylabel("Residuals")
ax1.set_title("Model 3", fontsize=22)

# Model 6
sns.residplot(x=model6.fittedvalues, y=model6.resid, lowess=True, line_kws={'color': 'red', 'lw': 2}, ax=ax2, scatter_kws={'color': viridis(2)})
ax2.axhline(0, color='black', linestyle='--', linewidth=1)
ax2.set_xlabel("Fitted Values")
ax2.set_title("Model 6", fontsize=22)

plt.tight_layout()
plt.show()

# Export figure to jpg
fig.savefig('figures/residuals_vs_fitted_values.jpg', dpi=300)

For both models, the residuals appear scattered randomly around the horizontal line at zero, suggesting that the model fits the data reasonably well in terms of linearity. There is no strong systematic curvature (e.g., U-shape or inverted U-shape) indicating a violation of linearity.

There are some points with very high residuals (above 30 or below -30). These might be potential outliers, which could influence the model. It would be worth investigating these points further.

The spread of residuals appears fairly consistent across the range of fitted values, suggesting no clear evidence of heteroscedasticity (non-constant variance of errors). However, this should be formally tested using tests like Breusch-Pagan.

**The linearity assumption seems reasonably satisfied, as there is no clear pattern or curvature in the residuals. There is a need to investigate points with high residuals to ensure they are not unduly affecting the models.**

In [ ]:
# ========================
# === Homoscedasticity ===
# ========================
# Check residuals vs fitted values and perform Breusch-Pagan test
from statsmodels.stats.diagnostic import het_breuschpagan

# Model 3
bp_test = het_breuschpagan(model3.resid, model3.model.exog)
labels = ['Lagrange multiplier statistic', 'p-value']
print(dict(zip(labels, bp_test)))

# Model 6
bp_test = het_breuschpagan(model6.resid, model6.model.exog)
labels = ['Lagrange multiplier statistic', 'p-value']
print(dict(zip(labels, bp_test)))

The Breusch-Pagan test is used to check for heteroscedasticity in a regression model, i.e., whether the variance of the residuals is constant across the range of fitted values (homoscedasticity).

Both Lagrange multiplier (LM) p-values and far below the conventional threshold (0.05). This indicates strong evidence to reject the null hypothesis of homoscedasticity, meaning that there is heteroscedasticity in the residuals of both models.

These concerns can be mitigated by using heteroscedasticity-robust standard errors to correct the issue without altering the coefficients themselves. I use Heteroscedasticity-Consistent Estimator 3, which uses the squared residuals as weights to adjust the variance-covariance matrix. Additionally, on the contrary of HC0, HC3 inflates the residual-based weights more for observations with high leverage, providing a more reliable correction when influential points exist.

In [ ]:
# ==============================
# === Normality of residuals ===
# ==============================
# Check QQ plot and Shapiro-Wilk test
from scipy.stats import shapiro, probplot
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Model 3
probplot(model3.resid, dist="norm", plot=ax1)
ax1.set_title("Model 3", fontsize=22)
ax1.get_lines()[0].set_color(viridis(1))

# Model 6
probplot(model6.resid, dist="norm", plot=ax2)
ax2.set_title("Model 6", fontsize=22)
ax2.set_ylabel("")
ax2.get_lines()[0].set_color(viridis(1))

plt.tight_layout()
plt.show()

# Save figure to jpg
fig.savefig('figures/qq_plots.jpg', dpi=300)

# Shapiro-Wilk Test for Model 3
shapiro_test_m3 = shapiro(model3.resid)
print(f"Shapiro-Wilk Test (Model 3): W={shapiro_test_m3[0]}, p-value={shapiro_test_m3[1]}")

# Shapiro-Wilk Test for Model 6
shapiro_test_m6 = shapiro(model6.resid)
print(f"Shapiro-Wilk Test (Model 6): W={shapiro_test_m6[0]}, p-value={shapiro_test_m6[1]}")

In [ ]:
# =========================
# === Multicollinearity ===
# =========================
# Calculate VIF for each predictor
from statsmodels.stats.outliers_influence import variance_inflation_factor

# MODEL 3
# Get the exogenous variables (X) and their names
X = model3.model.exog
exog_names = model3.model.exog_names

# Variables to exclude
exclude_vars = ["immigrant:mreason[T.N/A]", "pow(age_dv, 2)"]
exclude_indices = [exog_names.index(var) for var in exclude_vars if var in exog_names]

# Remove the excluded variables from X and exog_names
X_filtered = pd.DataFrame(X).drop(columns=exclude_indices).values
exog_names_filtered = [name for i, name in enumerate(exog_names) if i not in exclude_indices]

# Calculate VIF for the remaining variables
vif_data = pd.DataFrame()
vif_data["Variable"] = exog_names_filtered
vif_data["VIF"] = [variance_inflation_factor(X_filtered, i) for i in range(X_filtered.shape[1])]

print("Variance Inflation Factor (VIF) - Model 3:")
print(vif_data)

# MODEL 6
X6 = model6.model.exog
exog_names6 = model6.model.exog_names

exclude_indices6 = [exog_names6.index(var) for var in exclude_vars if var in exog_names6]

X_filtered6 = pd.DataFrame(X6).drop(columns=exclude_indices6).values
exog_names_filtered6 = [name for i, name in enumerate(exog_names6) if i not in exclude_indices6]

vif_data6 = pd.DataFrame()
vif_data6["Variable"] = exog_names_filtered6
vif_data6["VIF"] = [variance_inflation_factor(X_filtered6, i) for i in range(X_filtered6.shape[1])]

print("Variance Inflation Factor (VIF) - Model 6:")
print(vif_data6)


For both models, both the Q-Q plot and Shapiro-Wilk test indicate that their residuals do not meet the normality assumption. While OLS is robust to some deviations from normality, severe deviations (as seen here) may affect the reliability of your hypothesis tests and confidence intervals.

Sometimes however, one can validly get away with non-normal residuals in an OLS context; see for example, Lumley T, Emerson S. (2002)

In [ ]:
# =================================
# === Check for leverage points ===
# =================================
# Calculate Cook's distance
#suppress scientific notation
import numpy as np
np.set_printoptions(suppress=True)

# Model 3
influence3 = model3.get_influence()

# Obtain Cook's distance for each observation
cooks = influence3.cooks_distance[0]

# Plot Cook's distance
plt.figure(figsize = (12, 8))
plt.stem(np.arange(len(cooks)), cooks)

# Model 6
influence6 = model6.get_influence()

# Obtain Cook's distance for each observation
cooks6 = influence6.cooks_distance[0]

# Plot Cook's distance
plt.figure(figsize = (12, 8))
plt.stem(np.arange(len(cooks6)), cooks6)

## V/ Robustness analysis

### Robustness to different immigration 'recency' thresholds

R has been used (survey package) to create 5 models at different thresholds determining whether an individual migrated 'recently' or not. I visualise the results here.

In [ ]:
# Load csv of results
years_robustness_df = pd.read_csv('tables/years_robustness_results.csv')

In [ ]:
years_robustness_df.columns

In [ ]:
# Create the plot
plt.figure(figsize=(10, 6))

# Plot the estimates as points
plt.scatter(years_robustness_df['threshold'], years_robustness_df['estimate'], 
           color=viridis(6), s=50, zorder=2)

# Add error bars for confidence intervals without vertical lines
plt.errorbar(years_robustness_df['threshold'], years_robustness_df['estimate'],
            yerr=[years_robustness_df['estimate'] - years_robustness_df['conf.low'],
                  years_robustness_df['conf.high'] - years_robustness_df['estimate']],
            fmt='o', color=viridis(6), alpha=0.5, capsize=3)

# Add reference line at y=0
plt.axhline(y=0, color='red', linestyle='-', linewidth=0.5)

# Customize the plot
plt.xlabel('Threshold', size = 15)
plt.ylabel('Estimate', size = 15)
plt.xticks([5, 10, 15, 20, 25], size = 14)  # Set x-axis ticks and labels
plt.yticks(size = 14)
plt.grid(True, axis='y', alpha=0.3)  # Only horizontal grid

# Export figure to jpg
plt.savefig('figures/years_robustness.jpg', dpi=300)